In [7]:
import fsspec
import xarray as xr

In [2]:
fs = fsspec.filesystem(
    protocol="az",
    account_name="saenigmaarchivedev",
    anon=False,
)

In [3]:
urls = list(fs.glob("analysis/uch/nwp2/*/*/*.nc"))

In [6]:
content = list()
for url in urls:
    with fs.open(url) as f:
        content.append(f.read())

In [10]:
xr.open_dataset(content[0])

<xarray.Dataset>
Dimensions:                            (y: 1606, x: 1906)
Coordinates:
    calculation_time_utc               datetime64[ns] ...
    time_utc                           datetime64[ns] ...
Dimensions without coordinates: y, x
Data variables:
    latitude                           (y, x) float64 ...
    longitude                          (y, x) float64 ...
    temperature_K                      (y, x) float32 ...
    wind_u_m_s                         (y, x) float32 ...
    wind_v_m_s                         (y, x) float32 ...
    accumulated_direct_radiation_W_m2  (y, x) float32 ...
    accumulated_global_radiation_W_m2  (y, x) float32 ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             ekmi
    GRIB_centreDescription:  Copenhagen
    GRIB_subCentre:          255
    Conventions:             CF-1.7
    institution:             Copenhagen
    history:                 2024-09-24T12:44 GRIB to CDM+CF via cfgrib-0.9.1...
    parameter:               temperature

In [147]:
import kerchunk.netCDF3

singles = []
for url in urls:
    with fs.open(url) as f:
        chunks = kerchunk.netCDF3.NetCDF3ToZarr(f"az://{url}", storage_options=fs.storage_options, inline_threshold=0, version=2)
        singles.append(chunks.translate())
    print("Done:", url)

Done: analysis/uch/nwp2/model_run_time_utc=20240922T150000Z/time_utc=20240922T150000Z/20240922T150000Z_20240922T150000Z.nc
Done: analysis/uch/nwp2/model_run_time_utc=20240922T150000Z/time_utc=20240922T160000Z/20240922T150000Z_20240922T160000Z.nc


In [148]:
from kerchunk.combine import MultiZarrToZarr

In [149]:
mzz = MultiZarrToZarr(
    singles,
    remote_protocol="az",
    remote_options=fs.storage_options,
    concat_dims=["time_utc"],
    identical_dims=["x", "y"]
)

In [150]:
out = mzz.translate()

In [151]:
import xarray as xr

In [152]:
ds = xr.open_dataset(
    "reference://",
    engine="zarr",
    backend_kwargs=dict(
        storage_options=dict(
            fo=out,
            remote_protocol="az",
            remote_options=fs.storage_options
        ),
        consolidated=False,
    ),
)

In [153]:
ds

<xarray.Dataset>
Dimensions:                            (time_utc: 1, y: 1606, x: 1906)
Coordinates:
    calculation_time_utc               (time_utc) datetime64[ns] ...
  * time_utc                           (time_utc) datetime64[ns] 2024-09-22T1...
Dimensions without coordinates: y, x
Data variables:
    accumulated_direct_radiation_W_m2  (time_utc, y, x) float32 ...
    accumulated_global_radiation_W_m2  (time_utc, y, x) float32 ...
    latitude                           (time_utc, y, x) float64 ...
    longitude                          (time_utc, y, x) float64 ...
    temperature_K                      (time_utc, y, x) float32 ...
    wind_u_m_s                         (time_utc, y, x) float32 ...
    wind_v_m_s                         (time_utc, y, x) float32 ...
Attributes:
    Conventions:             CF-1.7
    GRIB_centre:             ekmi
    GRIB_centreDescription:  Copenhagen
    GRIB_edition:            2
    GRIB_subCentre:          255
    history:                 2024-09-24T12:44 GRIB to CDM+CF via cfgrib-0.9.1...
    institution:             Copenhagen
    parameter:               temperature

In [11]:
import xarray as xr

In [12]:
ds = xr.open_dataset("local.h5")

In [13]:
ds

<xarray.Dataset>
Dimensions:                            (y: 1606, x: 1906)
Coordinates:
    calculation_time_utc               datetime64[ns] ...
    time_utc                           datetime64[ns] ...
    horizon                            timedelta64[ns] ...
Dimensions without coordinates: y, x
Data variables:
    latitude                           (y, x) float64 ...
    longitude                          (y, x) float64 ...
    temperature_K                      (y, x) float32 ...
    wind_u_m_s                         (y, x) float32 ...
    wind_v_m_s                         (y, x) float32 ...
    accumulated_direct_radiation_W_m2  (y, x) float32 ...
    accumulated_global_radiation_W_m2  (y, x) float32 ...
    number                             int32 ...
    step                               timedelta64[ns] ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             ekmi
    GRIB_centreDescription:  Copenhagen
    GRIB_subCentre:          255
    Conventions:             CF-1.7
    institution:             Copenhagen
    history:                 2024-09-23T08:33 GRIB to CDM+CF via cfgrib-0.9.1...
    parameter:               temperature